In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from keras.layers import Lambda, Input, Dense
from keras.models import Model
from keras.losses import mse
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
batch_size = 50
intermediate_dim = 100
epochs = 15
latent_dim = 5

In [ ]:
def sampling(args):
    z_mean, z_log_sigma = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(z_log_sigma) * epsilon

z = Lambda(sampling)([z_mean, z_log_sigma])

inputs = Input(batch_shape=(batch_size, original_dim), name='encoder_input')
h = Dense(intermediate_dim, activation='relu')(inputs)
z_mean = Dense(latent_dim, name='z_mean')(h)
z_log_sigma = Dense(latent_dim, name='z_log_sigma')(h)

encoder = Model(inputs, [z_mean, z_log_sigma, z]], name='encoder')
encoder.summary()

latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
decoder_h = Dense(intermediate_dim, activation='relu')(latent_inputs)
outputs = Dense(original_dim, activation='sigmoid')(decoder_h)

decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()

vae = Model(inputs, outputs, name='vae_mlp')

def vae_loss(x, x_decoded_mean):
    reconstruction_loss = mse(inputs, outputs) * original_dim
    kl_loss = - 0.5 * K.mean(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma), axis=-1)
    return reconstruction_loss + kl_loss

vae.compile(optimizer='rmsprop', loss=vae_loss)
vae.summary()

In [ ]:
vae.fit(x_train, x_train,
        shuffle=True,
        epochs=,
        batch_size=,
        validation_data=(x_test, x_test))